In [28]:
import pandas as pd
import seaborn as sns
import numpy as np
import calendar
from datetime import datetime

In [29]:
customers = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')

In [30]:
# Задача 1
# Сколько у нас пользователей, которые совершили покупку только один раз?
# Чтобы рассчитать кол-во уникальных покупателей, нужно к таблице orders приджойнить таблицу users. затем отфильтровать заказы
# created - они только созданы и не оплачены
# выделить уникальных пользователей и их количество заказов, отфильтровать тех, кто сделал только 1 заказ

first_task = (orders[~orders.order_status.isin(['created'])].merge( #left join по custer_id исклюяая статус created
        customers[['customer_id', 'customer_unique_id']], 
        on='customer_id', 
        how='left'
    )                                                         
    .groupby('customer_unique_id', as_index=False)
    .order_id.nunique()
    .query('order_id == 1')
    .customer_unique_id.nunique()
)

first_task # кол-во пользователей совершившие покупку 1 раз

93096

In [31]:
# Задача 2
# Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?
# отсеять заказы, с негативными событиями
# посчитать месяц
# посчитать уникальное количество заказов для каждого месяца и негативного события 
# рассчитать среднее количество заказов по месяцам для каждого статуса


second_task = (
    orders[orders.order_status.isin(['unavailable', 'canceled'])] #находим в данных неподтверждённые и отмененные заказы
        .assign(month = lambda row: row.order_purchase_timestamp.str[:7])  #добавляем колоку month
        .groupby(['month', 'order_status'], as_index=False) #группируем по новой колонке, и статусу заказа
        .order_id.nunique() #уникальные id заказов
        .groupby('order_status', as_index=False) #группировка по статусу заказов
        .order_id.mean()) #сортировка по среднему
second_task

,order_status,order_id
0,canceled,26.041667
1,unavailable,29.000000


In [32]:
# Задача 3
# джойн к items поля order_purchase_timestamp из orders
# Из поля order_purchase_timestamp получаем номер дня недели
# выделить дни, которому соответствет наибольшее количество продаж

third_task = (
    items
        .merge(
            orders[['order_id', 'order_purchase_timestamp']],
            on='order_id',
            how='left'
        )   #объединение двух фреймов данных на основе общего столбца
        .assign(dayofweek=lambda x: x.order_purchase_timestamp.apply(lambda y: parser.parse(y).weekday())) #добавляется новый столбец dayofweek
        .groupby('product_id', as_index=False) #таблица данных группируется по столбцу product_id
        .dayofweek
        .apply(lambda x: x.value_counts().index[0]) # для каждой группы (т.е. для каждого товара) вычисляется наиболее часто встречающийся день недели
    #первый элемент (index[0]) – это и будет самым часто встречающимся днем недели
        .rename(columns={'dayofweek': 'most_common_dayofweek'}) # переименовывается столбец dayofweek в most_common_dayofweek.
)
third_task.head()

,product_id,most_common_dayofweek
0,00066f42aeeb9f3007548bb9d3f33c38,6
1,00088930e925c41fd95ebfe695fd2655,1
2,0009406fd7479715e4bef61dd91f2462,3
3,000b8f95fcb9e0096488278317764d19,2
4,000d9be29b5207b54e86aa1b1ac54872,1


In [33]:
# Задача 4
# Создаем датафрейм с датами в диапазоне между наименьшей и наибольшей датами из столбца order_purchase_timestamp в датафрейме orders
date_range = pd.date_range(start=orders.order_purchase_timestamp.min()[:10], end=orders.order_purchase_timestamp.max()[:10])
calendar = pd.DataFrame(date_range, columns=['dates'])

# Создаем столбец month, содержащий месяцы для каждой даты в столбце dates
calendar['month'] = calendar.dates.apply(lambda x: x.isoformat()[:7])

# Разбиваем даты по месяцам и считаем количество уникальных дат в каждом месяце
calendar = calendar.groupby('month', as_index=False).dates.nunique()

# Рассчитываем количество недель в месяце
calendar['weeks_count'] = calendar.dates / 7

# Выбираем только те строки из датафрейма orders, у которых статус не равен 'created'
filtered_orders = orders[~orders.order_status.isin(['created'])]

# Объединяем этот датафрейм с датафреймом users, указывая столбец customer_id в качестве общего столбца
task_4 = filtered_orders.merge(customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Создаем столбец month, содержащий месяцы для каждой даты в столбце order_purchase_timestamp
task_4['month'] = task_4.order_purchase_timestamp.str[:7]

# Разбиваем даты по месяцам и клиентам и считаем количество уникальных заказов для каждой группы
task_4 = task_4.groupby(['customer_unique_id', 'month'], as_index=False).order_id.nunique()

# Объединяем этот датафрейм с датафреймом calendar, указывая столбец month в качестве общего столбца
task_4 = task_4.merge(calendar, on='month', how='left')

# Рассчитываем частоту заказов в неделю для каждого клиента
task_4['orders_weekly_frequency'] = task_4.order_id / task_4.weeks_count

# Выбираем только нужные столбцы
task_4 = task_4[['customer_unique_id', 'month', 'orders_weekly_frequency']]

# Выводим первые 5 строк
task_4.head(5)



,customer_unique_id,month,orders_weekly_frequency
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,0.233333


In [34]:
# Задача 5
task5 = orders[~orders.order_status.isin(['created'])].merge(
            customers[['customer_id', 'customer_unique_id']], 
            on='customer_id', 
            how='left'
        )   #Получим все заказы, у которых статус не равен "created", а затем соединим их с таблицей пользователей по полю "customer_id"

task5['month'] = task5['order_purchase_timestamp'].apply(lambda x: x[:7]) # Добавим в таблицу столбец "month", который содержит месяц, в который был сделан заказ.

task5 = task5.merge(
    task5.groupby('customer_unique_id', as_index=False).month.min().rename({'month': 'cohort'}, axis=1),
    on='customer_unique_id',
    how='left'
)  # Соединим таблицу с таблицей, содержащей месяц первого заказа для каждого пользователя.
 
task5['ret_month'] = task5.groupby('customer_unique_id', as_index=False).month.cumcount() #Добавим в таблицу столбец "ret_month", содержащий количество месяцев, прошедших с месяца первого заказа. 
task5['retention'] = 1 # Добавим столбец "retention", в котором все значения равны 1.

task5 = pd.pivot_table(task5, values='retention', index='cohort', columns='ret_month', aggfunc=np.sum) #Преобразуем таблицу, чтобы строки соответствовали месяцам первых заказов, столбцы - количеству месяцев, прошедших с месяца первого заказа, а значения - количеству покупателей, совершивших заказ в этом месяце.
task5 = task5.div(5, axis=0) # таблица "task5" будет содержать абсолютные значения количества покупателей, совершивших заказ в каждом месяце
task5

ret_month,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
cohort,,,,,,,,,,,,,,,,,
2016-09,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-10,64.2,2.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12,0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01,152.8,11.6,2.0,0.4,0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02,350.4,14.6,1.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03,527.2,26.0,3.4,0.6,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04,470.4,21.6,2.0,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-05,719.2,39.6,5.6,1.2,0.6,0.6,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
2017-06,627.8,34.6,2.6,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Задача 6
# функция для вычисления столбцов RFM
def get_rfm_columns(df, recency_col, frequency_col, monetary_col):
  intervals_r = list(df[recency_col].describe())[3:]
  intervals_f = list(df[frequency_col].describe())[3:]
  intervals_m = list(df[monetary_col].describe())[3:]
# Добавим столбцы RFM в фрейм данных, используя интервалы для привязки значений
  df['recency'] = df[recency_col].apply(lambda x: np.digitize(x, intervals_r))
  df['frequency'] = df[frequency_col].apply(lambda x: np.digitize(x, intervals_f))
  df['monetary'] = df[monetary_col].apply(lambda x: np.digitize(x, intervals_m))
# Возвращаем фрейм данных только со столбцами RFM и уникальным идентификатором клиента
  return df[['customer_unique_id', 'recency', 'frequency', 'monetary']]
# Рассчитаем количество дней от максимальной отметки времени покупки
max_date = parser.parse(orders.order_purchase_timestamp.max())
orders['days_to_max'] = (max_date - orders.order_purchase_timestamp.apply(parser.parse)).dt.days
# Вычислим столбцы RFM для фрейма данных заказов
task6 = (
    orders[~orders.order_status.isin(['created'])] # Исключаем заказы со статусом "создано"
        .merge(
            customers[['customer_id', 'customer_unique_id']], 
            on='customer_id', 
            how='left'
        ) # Присоединяем к фрейму данных пользователей
        .merge(
            items  # Присоединяем к фрейму данных items
                .groupby('order_id', as_index=False)  # Группируем по идентификатору заказа
                .price.sum() # Вычислим общую цену за заказ
        )
        .groupby('customer_unique_id', as_index=False) # Группировка по уникальному идентификатору клиента
        .agg({'order_id': lambda x: x.nunique(), 'days_to_max': np.min, 'price': np.sum})
) # Вычисляем значения RFM

task6 = get_rfm_columns(task6, 'days_to_max', 'order_id', 'price') # Используем функцию для вычисления столбцов RFM


In [27]:
task6

,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,1,4,3
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2,4,1
2,0000f46a3911fa3c0805444483337064,4,4,2
3,0000f6ccb0745a6a4b88665a16c9f078,3,4,1
4,0004aac84e0df4da2b147fca70cf8255,3,4,4
...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,4,4,4
95416,fffea47cd6d3cc0a88bd621562a9d061,3,4,2
95417,ffff371b4d645b6ecea244b27531430a,4,4,3
95418,ffff5962728ec6157033ef9805bacc48,2,4,3
